In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
import numpy as np

2023-11-18 10:50:18.975529: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-18 10:50:19.012551: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def scale_dataset(dataframe: pd.DataFrame, oversample: bool = False):
  # if target column is the last value
  X = dataframe[dataframe.columns[:-1]].values
  y = dataframe[dataframe.columns[-1]].values

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  if oversample:
      ros = RandomOverSampler()
      X, y = ros.fit_resample(X, y)

  data = np.hstack((X, np.reshape(y, (-1, 1))))
  return data, X, y

In [1]:
def plot_loss(history):
  fig, ax = plt.subplots(nrows=1, ncols=1)
  ax.plot(history.history["loss"], label="loss")
  ax.plot(history.history["val_loss"], label="val_loss")
  ax.set_xlabel("Epoch")
  ax.set_ylabel("Binary crossentropy")
  ax.legend()
  ax.grid(True)
  plt.show()
  # fig.savefig("./out/loss.png")
  # plt.close()

def plot_accuracy(history):
  fig, ax = plt.subplots(nrows=1, ncols=1)
  ax.plot(history.history["accuracy"], label="accuracy")
  ax.plot(history.history["val_accuracy"], label="val_accuracy")
  ax.set_xlabel("Epoch")
  ax.set_ylabel("Accuracy")
  ax.legend()
  ax.grid(True)
  plt.show()
  # fig.savefig("./out/accuracy.png")
  # plt.close()

In [6]:
print("Loading dataset...")
df = pd.read_excel("./MachineLearning-Dataset-V1.xlsx")

# Train (60%), validation (20%) and test (20%) datasets
train, valid, test = np.split(df.sample(frac=1), [int(0.6 * len(df)), int(0.8 * len(df))])

train, X_train, y_train = scale_dataset(train, oversample=True)
valid, X_valid, y_valid = scale_dataset(valid, oversample=False)
test, X_test, y_test = scale_dataset(test, oversample=False)

nn_model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation="relu", input_shape=(9,)),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid"),
])

nn_model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001), 
    loss="binary_crossentropy", 
    metrics=["accuracy"])

history = nn_model.fit(
    X_train, y_train, epochs=100, batch_size=32, validation_split=0.2
)
# verbose=0

Loading dataset...
Epoch 1/100


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


243/243 [==============================] - 1s 2ms/step - loss: -26.8998 - accuracy: 0.4167 - val_loss: -382.1088 - val_accuracy: 0.0000e+00
Epoch 2/100
243/243 [==============================] - 0s 1ms/step - loss: -542.0864 - accuracy: 0.4167 - val_loss: -4293.4941 - val_accuracy: 0.0000e+00
Epoch 3/100
243/243 [==============================] - 0s 1ms/step - loss: -2648.1826 - accuracy: 0.4167 - val_loss: -15575.4287 - val_accuracy: 0.0000e+00
Epoch 4/100
243/243 [==============================] - 0s 1ms/step - loss: -7309.5273 - accuracy: 0.4167 - val_loss: -37436.5312 - val_accuracy: 0.0000e+00
Epoch 5/100
243/243 [==============================] - 0s 1ms/step - loss: -15227.2568 - accuracy: 0.4167 - val_loss: -71767.0234 - val_accuracy: 0.0000e+00
Epoch 6/100
243/243 [==============================] - 0s 1ms/step - loss: -26905.3926 - accuracy: 0.4167 - val_loss: -120095.7031 - val_accuracy: 0.0000e+00
Epoch 7/100
243/243 [==============================] - 0s 1ms/step - loss: -427

In [2]:
plot_loss(history)
plot_accuracy(history)

NameError: name 'history' is not defined